In [145]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

attr_cols = ['crossing', 'finishing', 'heading_accuracy','short_passing', 'volleys', 'dribbling', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_positioning','gk_reflexes','form']

x_cols = [ "%s_%s"%(c,attr)  for c in ['h','a'] for attr in attr_cols]
x_norm_cols = [ "%s_%s"%(c,attr)  for c in ['h','a'] for attr in attr_cols if attr!='form']
y_cols = ['Result']

full_data = pd.read_csv('Full_Data_feats_ha.csv' )
#Finding misclassifications

def misclassification_report(df, y_pred, y_test ):
    
    misclassified = pd.DataFrame()
    
    for count,value  in enumerate(y_test.iterrows()):
        
        if y_pred[count]!=int(value[1]['Result']):
            new_row = df.iloc[value[0]]
            new_row['Predicted'] = y_pred[count]
            misclassified = misclassified.append(new_row)
    return misclassified


In [146]:
def normalize(df):
    for key in df:
        if key in x_norm_cols:
            mn = df[key].min()
            mx = df[key].max()
            diff = mx - mn
            df[key] = df[key].apply(lambda x : (x-mn)/diff)
    return df

tr_data = normalize(full_data)
#take difference and store data 

for i in attr_cols:
    tr_data['%s'%i] = tr_data["h_%s"%i] - tr_data["a_%s"%i]
    
tr_data_diff = tr_data[attr_cols]
print(tr_data_diff.columns)

Index(['crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'free_kick_accuracy', 'long_passing', 'ball_control',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'marking', 'standing_tackle', 'sliding_tackle', 'gk_positioning',
       'gk_reflexes', 'form'],
      dtype='object')


In [147]:
num_features = 10
from sklearn.feature_selection import f_classif, SelectKBest

sel_feats = SelectKBest(k = num_features).fit_transform(tr_data_diff,full_data[y_cols])
print(sel_feats.shape)

x_all = tr_data_diff
y_all = full_data[y_cols]

(1140, 10)


D:\Apps\Anaconda3\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [148]:
# from sklearn.decomposition import PCA
# comps = 10
# pca = PCA(n_components=comps, svd_solver='full')
# x_pca = pca.fit_transform(x_all)  
# print(x_pca.shape)

import numpy as np
#One hot encoding the y values 
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=False)
results = np.array(full_data['Result'])
y_all = onehot_encoder.fit_transform(results.reshape(-1,1))

D:\Apps\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [155]:
x_train,x_test,y_train,y_test = train_test_split(x_all, y_all, test_size = 0.33, random_state = 42, stratify = y_all)





In [192]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout


def create_model2(x_train,y_train):
    """ Creates a Sequential model made out of Densely connected Neural Network layers. All parameters are configured as per specification for Q4"""

    model = Sequential()
    #Input Layer with 61 inputs corresponding to 61 features 
    model.add(Dense(10,input_dim = 30, activation = 'relu'))

    model.add(Dense(100,activation = 'relu'))
    model.add(Dense(700,activation = 'relu'))
    model.add(Dense(123,activation = 'sigmoid'))
    model.add(Dense(3,activation = 'softmax'))
    
    #Compiling the model with appropriate parameters
    model.compile(loss = 'mse', optimizer ='adam',metrics = ['accuracy'])

    result = model.fit(x_train,y_train,epochs = 10, batch_size = 100)

    #Store the model, its average training accuracy and validation accuracy over 10 epochs

    return model

def create_model(x_train,y_train):
    """ Creates a Sequential model made out of Densely connected Neural Network layers. All parameters are configured as per specification for Q4"""

    model = Sequential()
    #Input Layer with 61 inputs corresponding to 61 features 
    model.add(Dense(100,input_dim = 30, activation = 'relu'))

    model.add(Dense(500,activation = 'relu'))
    model.add(Dense(200,activation = 'sigmoid'))
    model.add(Dense(3,activation = 'softmax'))
    
    #Compiling the model with appropriate parameters
    model.compile(loss = 'mse', optimizer ='nadam',metrics = ['accuracy'])

    result = model.fit(x_train,y_train,epochs = 10, batch_size = 100)

    #Store the model, its average training accuracy and validation accuracy over 10 epochs

    return model

#Training neural network
print(len(x_train),len(x_test))
model = create_model(x_train,y_train)
model2 = create_model2(x_train,y_train)

print(y_test)

pred_y1 = model.predict(x_test)
pred_y2 = model2.predict(x_test)
print(len(pred_y1))
print(pred_y1,pred_y2)

763 377
Epoch 1/10
763/763 [==============================] - 2s 3ms/step - loss: 0.2128 - acc: 0.4404
Epoch 2/10
763/763 [==============================] - 0s 108us/step - loss: 0.2014 - acc: 0.5308
Epoch 3/10
763/763 [==============================] - 0s 105us/step - loss: 0.1934 - acc: 0.5452
Epoch 4/10
763/763 [==============================] - 0s 128us/step - loss: 0.1910 - acc: 0.5596
Epoch 5/10
763/763 [==============================] - 0s 121us/step - loss: 0.1876 - acc: 0.5518
Epoch 6/10
763/763 [==============================] - 0s 105us/step - loss: 0.1844 - acc: 0.5780
Epoch 7/10
763/763 [==============================] - 0s 110us/step - loss: 0.1823 - acc: 0.5531
Epoch 8/10
763/763 [==============================] - 0s 134us/step - loss: 0.1737 - acc: 0.6147
Epoch 9/10
763/763 [==============================] - 0s 116us/step - loss: 0.1717 - acc: 0.6134
Epoch 10/10
763/763 [==============================] - 0s 149us/step - loss: 0.1658 - acc: 0.6317
Epoch 1/10
763/763 [===

In [193]:
from sklearn.metrics import accuracy_score
new_pred = []

for i in range(0,len(pred_y1)):
    cur_pred = []
    for j in range(0,3):
        
        cur_pred.append((pred_y1[i][j] + pred_y2[i][j]))
    new_pred.append(cur_pred)
    
inv = onehot_encoder.inverse_transform(new_pred)

y_test_inv =  onehot_encoder.inverse_transform(y_test)

inv_pred = inv.reshape(1,-1)[0]

a = accuracy_score(y_test_inv,inv_pred)
print(a)
# #Training linear regression on the goal difference 
# from sklearn.linear_model import LinearRegression
# import math
# lr = LinearRegression()
# lr.fit(x_train,y_train['Goal_Diff'])
# y_pred = lr.predict(x_test[x_cols])
# goals = [math.ceil(value) for value in y_pred]

# pred_res = [1 if value > 0 else (-1 if value < 0 else 0) for value in goals]
# print(z)

0.5092838196286472


In [ ]:
#Train SVM
from sklearn.svm import LinearSVC, SVC

svm = LinearSVC(C = 0.25, loss = 'hinge',penalty = 'l2')
svm.fit(x_train,y_train)
# print(svm.coef_)
# print(svm.intercept_)
y_pred = svm.predict(x_test)
z =accuracy_score(y_test,y_pred)
print(z)
print(y_pred)
mc = misclassification_report(full_data,y_pred,y_test)
print(mc[x_cols + y_cols + ['Predicted']])


In [ ]:
#Training decision tree
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

dt = DecisionTreeClassifier(criterion = 'gini', max_depth = 4)

dt.fit(x_train,y_train)

y_pred = dt.predict(x_test)

#y_test_synth = [-1 if total < 0 else (1 if total > 0 else 0) for total in x_test['Total'].values] 

a = accuracy_score(y_test,y_pred)
print(a)
# print(te_y.values)
#mc = misclassification_report(full_data, y_pred,y_test)
#print(mc)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


knn = KNeighborsClassifier(n_neighbors=8)

knn.fit(x_train,y_train)

knn_pred = knn.predict(x_test)
ak = accuracy_score(x_test,knn_pred)

print(ak)
